# Segmentation QC

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [10]:
# macOS requirement
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import celldega as dega
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')

In [ ]:
metrics_df, gene_specific_metrics_df = dega.qc.qc_segmentation(transcript_metadata_file="data/segmentation_metrics_data/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_partitioned_transcripts_metadata.parquet", 
                                                          transcript_data_file="data/segmentation_metrics_data/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-create_subset_subset_coordinates.csv", 
                                                          cell_polygon_metadata_file="data/segmentation_metrics_data/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_cell_metadata.parquet", 
                                                          cell_polygon_data_file="data/segmentation_metrics_data/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_cell_polygons.parquet", 
                                                          subset_interval_y_x=[0,20294,0,42748],
                                                          pixel_size=0.2125, 
                                                          dataset_name="Xenium-Prostate", 
                                                          segmentation_approach="Cellpose2", 
                                                          from_stp = True)

segmentation metrics calculation completed


In [7]:
metrics_df

,proportion_transcripts_assigned_to_cells,total_number_of_cells,average_cell_area,average_cell_volume,average_transcripts_per_cell,median_transcripts_per_cell,average_genes_per_cell,median_genes_per_cell,cells_per_100_um^2,percent_empty_cells
Xenium-Prostate-Cellpose2,91.175765,222902.0,2326.244272,2326.244272,244.308589,175.0,179.442647,143.0,0.569,5.702634


In [8]:
gene_specific_metrics_df

,A2ML1,AAMP,AAR2,AARSD1,ABAT,ABCA1,ABCA3,ABCA4,ABCA7,ABCB1,...,ZPR1,ZSCAN1,ZSCAN12,ZSCAN16,ZSCAN20,ZSCAN26,ZSWIM6,ZUP1,ZYG11B,ZYX
metric_name,,,,,,,,,,,,,,,,,,,,,
proportion_of_cells_expressing_gene,0.002481,0.080183,0.023656,0.026509,0.065536,0.040614,0.010561,0.003037,0.007707,0.013199,...,0.044284,0.001243,0.006272,0.013804,0.003468,0.020296,0.012956,0.018461,0.023239,0.082045
average_expression_of_gene,123057.208754,121548.253464,121304.050548,121298.324476,120963.472938,123923.963961,122071.672635,124475.617479,124972.854767,127129.511957,...,122189.691219,122167.853242,123799.095175,123909.465307,123688.196341,122751.211645,124136.573004,124365.246235,121100.706963,123282.481808
assigned_transcripts_per_gene,0.950400,0.922834,0.922300,0.935246,0.929936,0.941423,0.936224,0.966759,0.943515,0.932544,...,0.906876,0.966997,0.877610,0.934869,0.925508,0.908332,0.946553,0.925840,0.890543,0.882710


In [9]:
cells_per_um2 should be calculated in the tissue region only (use alphashape)
add tissue alphashape in the metrics df as a column

In [4]:
import alphashape

In [5]:
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPoint, LineString
from shapely.affinity import translate

In [ ]:


#geoms = alpha_shape.geoms  
largest_polygon = max(alpha_shape.geoms, key=lambda p: p.area)

#geoms_with_area = [(geom, geom.area) for geom in geoms if isinstance(geom, Polygon)]
#sorted_geoms = sorted(geoms_with_area, key=lambda x: x[1], reverse=True)
#if len(sorted_geoms) > 1:
#    second_largest_geom = sorted_geoms[1][0]
#else:
#    second_largest_geom = None
    
fig, ax = plt.subplots(figsize=(40, 40))
ax.scatter(points[:, 0], points[:, 1], color='blue', label='Vertices', s=50)

x, y = largest_polygon.exterior.xy
ax.plot(x, y, color='red', label='Alpha Shape', linewidth=20)

ax.legend(fontsize=30)
plt.show()

gdf = gpd.GeoDataFrame(geometry=[largest_polygon])
gdf.to_parquet('../../../Downloads/alphashape_epidermis_skin.parquet', index=False)

In [8]:
trx_xen_def = pd.read_parquet("/Users/jishar/Documents/cell_segmentation/segmentation_data/original_data/Xenium_Prime_Human_Skin_FFPE_outs/transcripts.parquet")

In [11]:
tuple_list = list(zip(trx_xen_def['x_location'], trx_xen_def['y_location']))
points = np.array(tuple_list)

In [ ]:
alpha_value = 0.008
alpha_shape = alphashape.alphashape(points, alpha_value)
alpha_shape

In [ ]:
alpha_value = 1
alpha_shape = alphashape.alphashape(points, alpha_value)
alpha_shape